In [1]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator 
from keras import backend as K

Using TensorFlow backend.


In [2]:
img_rows,img_cols = 395, 700

VGG = VGG19(weights='imagenet',
            include_top=False,
            input_shape=(img_rows,img_cols,3))

for layer in VGG.layers:
    layer.trainable=False

def addTopModelVGG(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 7

FC_Head = addTopModelVGG(VGG,num_classes)

model = Model(inputs = VGG.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 395, 700, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 395, 700, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 395, 700, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 197, 350, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 197, 350, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 197, 350, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 98, 175, 128)      0   

In [3]:
train_data_dir = 'train'
validation_data_dir = 'test'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    #rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'VGG19_5.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1200
nb_validation_samples = 300  

epochs = 26
batch_size = 32

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)


Found 4165 images belonging to 7 classes.
Found 1785 images belonging to 7 classes.
Epoch 1/26
37/37 [==============================] - 1795s 49s/step - loss: 1.9710 - accuracy: 0.1443 - val_loss: 1.9280 - val_accuracy: 0.1389

Epoch 00001: val_loss improved from inf to 1.92796, saving model to VGG19_5.h5
Epoch 2/26
37/37 [==============================] - 1869s 51s/step - loss: 1.9470 - accuracy: 0.1470 - val_loss: 1.9535 - val_accuracy: 0.1424

Epoch 00002: val_loss did not improve from 1.92796
Epoch 3/26
37/37 [==============================] - 1865s 50s/step - loss: 1.9423 - accuracy: 0.1495 - val_loss: 1.9494 - val_accuracy: 0.2465

Epoch 00003: val_loss did not improve from 1.92796
Epoch 4/26
37/37 [==============================] - 1868s 50s/step - loss: 1.8628 - accuracy: 0.2272 - val_loss: 1.7498 - val_accuracy: 0.3090

Epoch 00004: val_loss improved from 1.92796 to 1.74976, saving model to VGG19_5.h5
Epoch 5/26
37/37 [==============================] - 1965s 53s/step - loss: 1